In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import json
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

import yaml
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/fasttext-sbwc.3.6.e20.vec'

In [2]:
cantidad = 100000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [3]:
path_to_json = 'output_seccionado/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))
        


In [4]:
#jsons = []
#for index, js in enumerate(json_files):
#    with open(os.path.join(path_to_json, js)) as json_file:
#        jsons.append(yaml.load(json_file, yaml.SafeLoader))
        

In [5]:
jsons[0]

{'nombre archivo': '1582718980347-CV_-_Mario_Espinoza1',
 'extras': 'mario sergio espinoza acero \n dni : 45629118 \n mariosergioespinoza89@gmail.com \n equipar fundamental empresa , orientar a resultar tangible . \n personal a cargar : 10 colaborador . \n servicio , , presupuesto gastos operaciones y gastos personal \n personal a cargar : 10 colaborador . \n servicio , , presupuesto gastos operaciones y gastos personal \n presentación y venta servicio selección , outsourcing , intermediación , payroll , \n personal a cargo : 7 personar \n prospección punto venta lima metropolitana . \n nivel : avanzado \n ',
 'experiencia': 'persona proactiva , ordenar y alto sentir responsabilidad . \n experiencia laboral \n adecco peru \n cargo : gerente desarrollo \n fecha : 01/04/2019 a fecho . \n funciones : \n responsable manejar cartero top cliente . \n responsable cumplimiento objetivo venta líneo negociar , garantizar \n rentabilidad y nivel servicio \n desarrollo convenio diverso institución

In [6]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [7]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

0.8897964


In [8]:
get_closest(word= 'solidaridad', n = 3)

(['solidaridad', 'solidaria', 'insolidaridad', 'solidario'],
 [1, 0.5968679189682007, 0.5851333141326904, 0.5850723385810852])

In [9]:
model.similarity('educación', 'educar')

0.49962938

In [10]:
import es_core_news_md
nlp = es_core_news_md.load()
nlp_text = nlp('educado')
nlp_text[0].lemma_

'educar'

In [11]:
prc_description = '''ingeniería máster python excel desarrollo gestión comercial experiencia manejo clientes
emprendimiento liderar equipos planificar organizar dirigir trabajo presión seguimiento KPI machine learning
telecomunicaciones eléctrica licenciatura pytorch java computación'''

In [12]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        #print(similar_words[i], word_value[similar_words[i]])
        #print('------------------------------------------------')

In [13]:
word_value.keys()

dict_keys(['ingeniería', 'ingenierías', 'mecatrónica', 'máster', 'másteres', 'licenciatura', 'python', 'perl', 'scripts', 'excel', 'powerpoint', 'word', 'desarrollo', 'sostenible', 'socioeconómico', 'gestión', 'gestionar', 'planificación', 'comercial', 'comerciales', 'comercio', 'experiencia', 'experiencias', 'pericia', 'manejo', 'manejar', 'manejan', 'clientes', 'cliente', 'usuarios', 'emprendimiento', 'emprendimientos', 'emprendedor', 'liderar', 'encabezar', 'lidera', 'equipos', 'equipo', 'clubes', 'planificar', 'planear', 'planificarse', 'organizar', 'organizarlo', 'dirigir', 'dirigirlo', 'dirigirla', 'trabajo', 'remunerado', 'trabajos', 'presión', 'presiones', 'sobrepresión', 'seguimiento', 'seguimientos', 'evaluación', 'kpi', 'machine', 'machines', 'rage', 'learning', 'learn', 'education', 'telecomunicaciones', 'telecomunicación', 'telefonía', 'eléctrica', 'eléctrico', 'eléctricas', 'licenciaturas', 'doctorado', 'pytorch', 'java', 'sumatra', 'computación', 'computacional', 'comput

In [14]:
#'Licencia' in jsons[1]['skills']

In [15]:
### ahora veamos si resulta el ranking
#frecuencia de término – frecuencia inversa de documento 
#Tf-idf
#Para calcular este ranking es mejor tener las secciones skills y experiencia
#con el fin de calcular esta metrica usando la ocurrencia de las palabras
# Tenemos todos los CV's y una descripción del cargo, a este descripcion del cargo
# tiene N palabras, le buscamos 2 palabras parecidas, generando una descripcion
# de N*2

# Usando esta nuevo set de palabras de descripcion, recorremos todos los cvs contando 
# la ocurrencia de estas palabras en cada documento, y luego se genera un ranking usando Tf-idf
# La pregunta es: ¿Lo haré sobre el documento entero? o ¿Trataré de seccionar y ocupar ciertas secciones?

no_of_cv = len(jsons)
print(no_of_cv)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        #jsons[i]['skills'] = [x.lower() for x in jsons[i]['skills']]
        try:
            #if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
            if word in jsons[i]['skills'] or word in jsons[i]['experiencia'] or word in jsons[i]['educación']:
                #print('entre')
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
    print(idf)

34
{'ingeniería': 0.581921545449721}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616, 'licenciatura': 2.4277482359480516}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.5263605246161616, 'mecatrónica': 3.5263605246161616, 'máster': 3.5263605246161616, 'másteres': 3.5263605246161616, 'licenciatura': 2.4277482359480516, 'python': 1.916922612182061}
{'ingeniería': 0.581921545449721, 'ingenierías': 3.526360

In [16]:

score = {}
for i in range(no_of_cv):
    score[i] = 0

    for word in word_value.keys():
        tf = jsons[i]['skills'].count(word) + jsons[i]['experiencia'].count(word) + jsons[i]['educación'].count(word)
        #print(tf)
        if (tf == 0):
            #print(jsons[i]['nombre archivo'])
            #print(word)
            tf = 1
        else:
            print('Pillé ' + word+ ' en' +jsons[i]['nombre archivo'] )
            #print(word)
        score[i] += word_value[word]*tf*idf[word]

print(score)

Pillé desarrollo en1582718980347-CV_-_Mario_Espinoza1
Pillé gestión en1582718980347-CV_-_Mario_Espinoza1
Pillé comercial en1582718980347-CV_-_Mario_Espinoza1
Pillé experiencia en1582718980347-CV_-_Mario_Espinoza1
Pillé manejar en1582718980347-CV_-_Mario_Espinoza1
Pillé cliente en1582718980347-CV_-_Mario_Espinoza1
Pillé seguimiento en1582718980347-CV_-_Mario_Espinoza1
Pillé kpi en1582718980347-CV_-_Mario_Espinoza1
Pillé ingeniería en1580920711029-Kevin_SQ
Pillé excel en1580920711029-Kevin_SQ
Pillé desarrollo en1580920711029-Kevin_SQ
Pillé gestión en1580920711029-Kevin_SQ
Pillé comercial en1580920711029-Kevin_SQ
Pillé comerciales en1580920711029-Kevin_SQ
Pillé experiencia en1580920711029-Kevin_SQ
Pillé manejo en1580920711029-Kevin_SQ
Pillé cliente en1580920711029-Kevin_SQ
Pillé lidera en1580920711029-Kevin_SQ
Pillé seguimiento en1580920711029-Kevin_SQ
Pillé evaluación en1580920711029-Kevin_SQ
Pillé kpi en1580920711029-Kevin_SQ
Pillé ingeniería en1581878742461-Curriculum_Joselyn_Mari_2020

In [17]:
jsons[0]['skills']

'marketing y gestión comercial . \n profesión : ciencias comunicación . \n '

In [18]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['nombre archivo']))
    
sorted_list.sort(reverse = True)

#for s, i in sorted_list:
#    if list(cvs)[i] != '.DS_Store':
#        print(list(cvs)[i], ':', s)

In [19]:
sorted_list

[(184.4120468405768, '1581023751928-CV'),
 (172.3922412823049, '1580913938357-CV_Gianfranco_Vela_ESP_Ene.2020'),
 (170.50237478447585, '1580320654107-CV_Gabriel_Garcia_Lorca'),
 (170.40221145193638, '1581954259745-CV_2020_ANTONIO_HARTMANN_R'),
 (170.2035921633964, '1581000554896-Nabib_M_Garcia_Hirsh_CV'),
 (170.0624039779352, 'prueba_genoma_experiencia'),
 (170.0624039779352, '1581878742461-Curriculum_Joselyn_Mari_2020_1'),
 (169.86823176894285, '1580248761709-CV_Sergio_Soto_Valdes'),
 (169.86547374404938, '1580920711029-Kevin_SQ'),
 (169.76810172388542, '1580497335052-Jose_Medina_13-1-2020'),
 (168.94681860539905, '1580573732905-DiegoAlbertoGutierrezBejarano'),
 (168.84851178455537, '1582313013581-CV_FRF'),
 (167.96653064782598, '1582640494849-ANDRES_MENA_CV'),
 (167.46928004364662,
  '1580231848251-Curriculum_Vitae_-_Jesús_Alonso_Vásquez_Cole'),
 (166.92370818940165, '1586903416737-NewHTJ'),
 (166.4656364588117, '1583796411230-Curriculum__fco_marzo'),
 (166.41664499411783, '158169055